In [ ]:
!pip install tqdm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re

Здесь подкачиваются данные с гугл диска для дальнейшей работы

In [ ]:
!gdown 1YHLbtohCZ4NxHeDMN0iDweV0ota7tQPC
!unzip train_dataset_vprod_train.zip

In [ ]:
# Попробуем прочитать файл в режиме бинарного чтения и декодировать с обработкой ошибок
with open('vprod_train/JOB_LIST.csv', 'rb') as file:
    content = file.read()

# Попробуем декодировать содержимое с заменой ошибок
decoded_content = content.decode('utf-8', errors='replace')

# Сохранение декодированных данных во временный файл для последующего чтения
with open('temp.csv', 'w', encoding='utf-8') as temp_file:
    temp_file.write(decoded_content)

# Чтение данных из временного файла
df = pd.read_csv('temp.csv')


In [ ]:
df = df.dropna()
df = df.drop_duplicates()

In [ ]:
# Функция для фильтрации только русских, английских букв и пробелов
def preprocess_text(text):
    # Оставляем только русские, английские буквы и пробелы
    text = text.lower()
    # text = re.sub(r'\d+', '', text)
    # text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^а-яА-Яa-zA-Z\s]', '', text)
    return text


# Применяем предобработку ко всем строкам в колонке 'job_title'
df['job_title'] = df['job_title'].apply(preprocess_text)


In [ ]:
sampled_df = df.sample(n=100000, random_state=42) #Случайная выборка 100 тысяч образцов

In [ ]:
# Векторизация профессий
vectorizer = TfidfVectorizer(analyzer='char')
X = vectorizer.fit_transform(sampled_df['job_title'])

In [ ]:
# Кластеризация
kmeans = KMeans(n_clusters=10000, random_state=42)
sampled_df['cluster'] = kmeans.fit_predict(X)

# Обобщение профессий
def get_general_profession(cluster):
    cluster_data = sampled_df[sampled_df['cluster'] == cluster]
    return cluster_data['lemmatized_profession'].mode()[0]

sampled_df['general_profession'] = sampled_df['cluster'].apply(get_general_profession)
# Замена в датафрейме
sampled_df['final_profession'] = sampled_df['general_profession']

In [ ]:
# Инициализация модели и токенизатора
model_name = 'gpt2'  # Вы можете выбрать другую модель, если хотите
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

# Функция для запроса к модели
def get_common_profession(cluster_examples):
    prompt = (
        "Собраны примеры профессий из одного кластера:\n"
        + "\n".join(cluster_examples) + "\n"
        + "Пожалуйста, объедините их в одно общее название профессии:"
    )

    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=100,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split('\n')[-1].strip()  # Возвращаем последний ответ после новой строки

# Сбор примеров из каждого кластера и получение объединенных названий профессий
clustered_professions = sampled_df.groupby('cluster')['job_title'].apply(list).to_dict()
new_professions = []

for cluster, examples in clustered_professions.items():
    # Получаем 5 примеров для каждого кластера
    cluster_examples = examples[:5]
    common_profession = get_common_profession(cluster_examples)
    new_professions.append({
        'cluster': cluster,
        'common_profession': common_profession
    })

# Создание нового DataFrame
new_df = pd.DataFrame(new_professions)

# Сохранение нового DataFrame в CSV
new_df.to_csv('common_professions.csv', index=False)

# Вывод результата
print(new_df)
